In [1]:
%run /spark-data/CRM/utilities/common_utility.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/13 06:23:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Initialize Spark Session

In [2]:
spark = initialize_spark_session("Transactions Cleaning")

24/09/13 06:23:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Logs Configuration

In [3]:
log_file_path = 'logs/transactions_cleaning.log'
logger = initialize_logger(log_file_path)

logger.info("Logger initialized with dynamic path!")

2024-09-13 06:23:45,848 - logger - INFO - Logger initialized with dynamic path!


# Dataset Load

In [4]:
transactions_path = "/spark-data/CRM/Dataset/transactions.csv"
transactions_df = load_data_files(transactions_path)
display_dataframes(transactions_df)

2024-09-13 06:23:48,726 - logger - INFO - Displayed first 5 records of Spark DataFrame.


+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|Transaction_ID                      |Customer_ID                         |Product_ID|Date      |Amount|Sales_Rep_ID                        |
+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|31190b6c-54cd-4cdb-a89e-7d8633b386df|d7522cab-f713-4c74-9d32-fadba3a20a85|48        |2024-04-01|234.0 |8aede4f3-bd43-45a7-9edc-5d69a2645fa1|
|05d4bcca-dc8e-405c-81aa-d2bfc2acbc00|dae0689d-0c38-440c-b921-fe2413c3df3b|3         |2024-03-28|119.0 |c48b42a3-544b-4542-8528-4208e6f80b46|
|9b6e01be-1814-41e0-bd7d-3ae2f845f5bf|075005e4-bb34-4965-bb28-fd100b10c45c|26        |2024-06-09|240.0 |d7558c8d-5355-48e7-8b88-30ab41ff2b8f|
|46bd8f7c-20e0-405b-a1fe-b78e2b9a6813|ec910504-7738-463c-b269-1b1617b09ad1|37        |2024-03-24|451.0 |d0fa26a0-d161-4976-ae9a-9c5f74fc103c|
|f02dd

# Data Preprocessing

In [5]:
# Step 1: Count missing values in each column before filling them
logger.info("Step 1: Counting missing values in each column before filling them...")
missing_values_before = count_missing_values(transactions_df)
missing_values_before.show()

# Step 2: Calculate the mean value of the 'Amount' column
logger.info("Step 2: Calculating the mean value of the 'Amount' column...")
mean_amount_row = transactions_df.agg(mean("Amount")).collect()[0]
mean_amount = mean_amount_row[0]
print(f"Mean value calculated for 'Amount': {mean_amount:.2f}")

# Step 3: Fill missing values in the 'Amount' column with the rounded mean value
logger.info("Step 3: Filling missing values in the 'Amount' column with the mean value...")
cleaned_transactions_df = fill_missing_values(transactions_df, {"Amount": round(mean_amount, 2)})

# Step 4: Check for duplicate records based on 'Transaction_ID'
logger.info("Step 4: Checking for duplicate records based on 'Transaction_ID'...")
cleaned_transactions_df = drop_duplicates(cleaned_transactions_df, "Transaction_ID")

# Step 5: Ensure 'Date' is in YYYY-MM-DD format
logger.info("Step 5: Ensuring 'Date' column is in YYYY-MM-DD format...")
cleaned_transactions_df = cleaned_transactions_df.withColumn(
    "Date", date_format(col("Date"), "yyyy-MM-dd")
)

# Step 6: date validation 
logger.info("Step 6: Validating dates present in Date column...")
cleaned_transactions_df = date_validation(cleaned_transactions_df,"Date")

# Step 7: Cross-verification after filling missing values in each column
logger.info("Step 7: Counting missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_transactions_df)
missing_values_after.show()

# Step 8: Correct inaccurate data (e.g., non-positive 'Amount')
logger.info("Step 8: Correcting non-positive values in the 'Amount' column...")
corrected_transactions_df = cleaned_transactions_df.withColumn(
    "Amount",
    when(col("Amount") <= 0, mean_amount).otherwise(col("Amount"))
)

# Step 9: Display the cleaned DataFrame
logger.info("Step 9: Displaying the cleaned 'transactions_df' DataFrame...")
corrected_transactions_df.show(5,truncate=False)

# Step 10: Save the cleaned data to a new CSV
logger.info("Step 10: Saving the cleaned data to 'cleaned_transactions.csv'...")
# save_df_to_csv(corrected_transactions_df, "Cleaned_data/cleaned_transactions.csv")

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_transactions_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")
logger.info("Data cleaning and export completed successfully.")

2024-09-13 06:23:48,740 - logger - INFO - Step 1: Counting missing values in each column before filling them...
2024-09-13 06:23:49,388 - logger - INFO - Step 2: Calculating the mean value of the 'Amount' column...


+--------------+-----------+----------+----+------+------------+
|Transaction_ID|Customer_ID|Product_ID|Date|Amount|Sales_Rep_ID|
+--------------+-----------+----------+----+------+------------+
|             0|          0|         0|   0|    52|           0|
+--------------+-----------+----------+----+------+------------+



2024-09-13 06:23:49,630 - logger - INFO - Step 3: Filling missing values in the 'Amount' column with the mean value...
2024-09-13 06:23:49,658 - logger - INFO - Step 4: Checking for duplicate records based on 'Transaction_ID'...


Mean value calculated for 'Amount': 282.78


2024-09-13 06:23:50,301 - logger - INFO - Number of duplicate records before dropping: 30
2024-09-13 06:23:50,740 - logger - INFO - Number of duplicate records after dropping: 0
2024-09-13 06:23:50,742 - logger - INFO - Step 5: Ensuring 'Date' column is in YYYY-MM-DD format...
2024-09-13 06:23:50,764 - logger - INFO - Step 6: Validating dates present in Date column...
2024-09-13 06:23:50,766 - logger - INFO - Step 1: Identifying future dates in 'Interaction_Date'...
2024-09-13 06:23:51,205 - logger - INFO - No future dates found.
2024-09-13 06:23:51,207 - logger - INFO - Step 7: Counting missing values in each column after filling them...
2024-09-13 06:23:51,861 - logger - INFO - Step 8: Correcting non-positive values in the 'Amount' column...
2024-09-13 06:23:51,891 - logger - INFO - Step 9: Displaying the cleaned 'transactions_df' DataFrame...


+--------------+-----------+----------+----+------+------------+
|Transaction_ID|Customer_ID|Product_ID|Date|Amount|Sales_Rep_ID|
+--------------+-----------+----------+----+------+------------+
|             0|          0|         0|   0|     0|           0|
+--------------+-----------+----------+----+------+------------+



2024-09-13 06:23:52,285 - logger - INFO - Step 10: Saving the cleaned data to 'cleaned_transactions.csv'...


+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|Transaction_ID                      |Customer_ID                         |Product_ID|Date      |Amount|Sales_Rep_ID                        |
+------------------------------------+------------------------------------+----------+----------+------+------------------------------------+
|00212675-af49-4cbd-9766-4303572b7506|41b14a02-df69-4f06-bba9-a32a32d7c008|30        |2024-05-01|125.0 |61989b73-1807-414a-a75b-f194c0152dba|
|003a04fd-3d61-4af1-bf78-64b4d8bec34b|2fbd8187-90f1-409e-89ff-3ff4cae288e6|14        |2024-01-23|50.0  |5184e796-6c9f-4019-907b-6e9e838430f3|
|007fa68d-e211-47af-85dc-e02f7477b5a9|006af455-013b-4c09-a6df-15ca3d41010f|43        |2024-02-14|357.0 |33ea8c7d-1b0a-4be7-9cff-d35ea357325f|
|00946fd1-cc54-4671-936d-7f6f25723e62|552651e9-3ff6-4c64-bdc5-863a8c65c5b2|28        |2024-01-19|448.0 |2f1183af-7ef6-498a-a851-91820338ff67|
|00b3f

2024-09-13 06:23:52,552 - logger - INFO - Data cleaning and export completed successfully.


Number of records after cleaning: 1000


In [6]:
spark.stop()